In [1]:
%pip install git+https://github.com/gaussalgo/adaptor
%pip install git+https://github.com/katarinagresova/AgoBind
%pip install sklearn
%pip install comet-ml
%pip install torchmetrics

  Cloning https://github.com/gaussalgo/adaptor to /tmp/pip-req-build-6_0yn149
  Running command git clone --filter=blob:none --quiet https://github.com/gaussalgo/adaptor /tmp/pip-req-build-6_0yn149
  Resolved https://github.com/gaussalgo/adaptor to commit db33e6e439babc68fe801a8946d87116ff44f170
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/katarinagresova/AgoBind to /tmp/pip-req-build-8k7msi3z
  Running command git clone --filter=blob:none --quiet https://github.com/katarinagresova/AgoBind /tmp/pip-req-build-8k7msi3z
  Resolved https://github.com/katarinagresova/AgoBind to commit ca59766c661cfc253745429111c9d0baeaa0b9d3
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel t

In [2]:
import comet_ml

In [3]:
# 1. pick the model base
from adaptor.lang_module import LangModule

kmer_len = 6
stride = 1
lang_module = LangModule(f"armheb/DNA_bert_{kmer_len}")

In [4]:
# 2. Initialize training arguments
# We apply NUM_STEPS stopping strategy in cases where at least one of the objectives does not converge in max_steps
from adaptor.utils import AdaptationArguments, StoppingStrategy

training_arguments = AdaptationArguments(output_dir="dnabert_for_clash",
                                         learning_rate=2e-4,
                                         weight_decay=0.01,
                                         max_steps=100000,
                                         stopping_strategy=StoppingStrategy.ALL_OBJECTIVES_CONVERGED,
                                         # stopping_strategy=StoppingStrategy.NUM_STEPS_ALL_OBJECTIVES,
                                         do_train=True,
                                         do_eval=True,
                                         warmup_steps=5000,
                                         gradient_accumulation_steps=4,
                                         logging_steps=100,
                                         eval_steps=100,
                                         save_steps=100,
                                         num_train_epochs=30,
                                         evaluation_strategy="steps",
                                         also_log_converged_objectives=True)

In [5]:
import pandas as pd
import numpy as np

def prepare_data(path_to_csv, path_to_txt, path_to_labels):
    dset = pd.read_csv(path_to_csv, sep='\t')
    dset['seq'] = dset.apply(lambda x: x['miRNA'] + 'NNNN' + x['gene'], axis=1)
    dset['seq'] = dset['seq'].apply(lambda x: ' '.join([x[i:i+kmer_len] for i in range(0, len(x)-kmer_len+1, stride)]))
    np.savetxt(path_to_txt, dset['seq'].values, fmt='%s')
    np.savetxt(path_to_labels, dset['label'].values, fmt='%s')

In [6]:
prepare_data('../data/train_set_1_1_CLASH2013_paper.tsv', '../data/train_set_1_1_CLASH2013_paper.txt', '../data/train_set_1_1_CLASH2013_paper_labels.txt')
prepare_data('../data/evaluation_set_1_1_CLASH2013_paper.tsv', '../data/evaluation_set_1_1_CLASH2013_paper.txt', '../data/evaluation_set_1_1_CLASH2013_paper_labels.txt')

In [7]:
from adaptor.evaluators.sequence_classification import SeqClassificationEvaluator, SeqClassificationEvaluator
from sklearn.metrics import precision_recall_curve, auc
#from torchmetrics import PrecisionRecallCurve
import torch
from transformers import PreTrainedTokenizer

from adaptor.evaluators.evaluator_base import EvaluatorBase
from adaptor.utils import Head, AdaptationDataset
    
class SequenceAucPr(SeqClassificationEvaluator):
    """
    Sequence classification area under Precision-Recall curve, where each input sample of dataset falls into a single category.
    """

    smaller_is_better: bool = False

    def __call__(self, model: torch.nn.Module, tokenizer: PreTrainedTokenizer, dataset: AdaptationDataset) -> float:
        """
        Refer to the superclass documentation.
        """
        expected = []
        actual = []

        for batch in dataset:
            expected.extend(batch["labels"].cpu())
            actual.extend(model(**batch).logits.argmax(-1).cpu())

        assert len(expected) == len(actual)

        #pr_curve = PrecisionRecallCurve(pos_label=1)
        p, r, thresholds = precision_recall_curve(expected, actual)
        auc_precision_recall = auc(r, p)
        return auc_precision_recall

In [8]:
# 3. pick objectives
# Objectives take either List[str] for in-memory iteration, or a source file path for streamed iterati
from adaptor.objectives.MLM import MaskedLanguageModeling
from adaptor.objectives.classification import SequenceClassification

mlm = MaskedLanguageModeling(lang_module,
                                 batch_size=16,
                                 texts_or_path='../data/train_set_1_1_CLASH2013_paper.txt',
                                 val_texts_or_path='../data/evaluation_set_1_1_CLASH2013_paper.txt',
                            )

cls = SequenceClassification(lang_module,
                                  batch_size=64,
                                  texts_or_path='../data/train_set_1_1_CLASH2013_paper.txt',
                                  labels_or_path='../data/train_set_1_1_CLASH2013_paper_labels.txt',
                                 val_texts_or_path='../data/evaluation_set_1_1_CLASH2013_paper.txt',
                                 val_labels_or_path='../data/evaluation_set_1_1_CLASH2013_paper_labels.txt',
                                 val_evaluators=[SequenceAucPr(decides_convergence=True)]
)

Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are n

In [9]:
# 4. pick a schedule of the selected objectives
# This one will initially fit the first objective until convergence on its eval set, then fits the second one 
from adaptor.schedules import ParallelSchedule, SequentialSchedule

schedule = SequentialSchedule([cls], training_arguments)
#schedule = ParallelSchedule([mlm, cls], training_arguments)
#schedule = SequentialSchedule(
#    objectives=[mlm, cls], 
#    args=training_arguments
#)

Total number of train samples: 30784
Total number of eval samples: 2000


In [10]:
comet_ml.init(project_name='dnabert_for_clash', api_key='3NQhHgMmmlfnoqTcvkG03nYo9')

COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /home/jovyan/.comet.config


In [ ]:
# 4. Run the training using Adapter, similarly to running HF.Trainer, only adding `schedule`
from adaptor.adapter import Adapter
#from transformers.integrations import CometCallback

adapter = Adapter(lang_module=lang_module, schedule=schedule, args=training_arguments)#, callbacks=[CometCallback()])
adapter.train()

max_steps is given, it will override any value given in num_train_epochs
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14430
  Num Epochs = 28
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 100000
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/katarinagresova/dnabert-for-clash/2beeacbecc194ac0ae1de8b72c521388

Automatic Comet.ml online logging enabled
Converged objective

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


SequenceClassification:  83%|████████▎ | 400/481 [00:49<00:09,  8.57batches/s, epoch=1, loss=0.699, split=train]Converged objectives: []
Evaluating...
***** Running Evaluation *****
  Num examples = 31
  Batch size = 1


{'loss': 0.737, 'learning_rate': 4.000000000000001e-06, 'train_SequenceClassification_loss': 0.73701636813581, 'train_SequenceClassification_num_batches': 400, 'epoch': 0.03}


SequenceClassification:  97%|█████████▋| 30/31 [00:01<00:00, 20.98batches/s, epoch=1, loss=0.701, split=eval]/opt/conda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:542: UserWarning: Length of IterableDataset <adaptor.utils.TransformerAdaptationDataset object at 0x7f748168acd0> was reported to be 31 (when accessing len(dataloader)), but 32 samples have been fetched. 
  warnings.warn(warn_msg)
SequenceClassification: 32batches [00:01, 21.34batches/s, epoch=1, loss=0.711, split=eval]                   
Converged objectives: []
tokenizer config file saved in dnabert_for_clash/checkpoint-100/SequenceClassification/tokenizer_config.json
Special tokens file saved in dnabert_for_clash/checkpoint-100/SequenceClassification/special_tokens_map.json
Configuration saved in dnabert_for_clash/checkpoint-100/SequenceClassification/config.json


...

Model weights saved in dnabert_for_clash/checkpoint-3500/SequenceClassification/pytorch_model.bin
SequenceClassification:  94%|█████████▍| 451/481 [00:58<00:03,  8.52batches/s, epoch=30, loss=0.0189, split=train]Converged objectives: []
Evaluating...
***** Running Evaluation *****
  Num examples = 31
  Batch size = 1


{'loss': 1.6123, 'learning_rate': 0.000144, 'train_SequenceClassification_loss': 1.8092812393018975, 'train_SequenceClassification_num_batches': 1000, 'epoch': 1.0}


SequenceClassification:  97%|█████████▋| 30/31 [00:01<00:00, 20.78batches/s, epoch=30, loss=0.0149, split=eval]/opt/conda/lib/python3.9/site-packages/torch/utils/data/dataloader.py:542: UserWarning: Length of IterableDataset <adaptor.utils.TransformerAdaptationDataset object at 0x7f746818bb20> was reported to be 31 (when accessing len(dataloader)), but 32 samples have been fetched. 
  warnings.warn(warn_msg)
SequenceClassification: 32batches [00:01, 21.21batches/s, epoch=30, loss=0.0149, split=eval]                   
Converged objectives: []
tokenizer config file saved in dnabert_for_clash/checkpoint-3600/SequenceClassification/tokenizer_config.json
Special tokens file saved in dnabert_for_clash/checkpoint-3600/SequenceClassification/special_tokens_map.json
Configuration saved in dnabert_for_clash/checkpoint-3600/SequenceClassification/config.json


{'eval_loss': 2.1321206092834473, 'eval_runtime': 1.5146, 'eval_samples_per_second': 20.468, 'eval_steps_per_second': 20.468, 'eval_SequenceClassification_loss': 1.973819205167375, 'eval_SequenceClassification_num_batches': 1152, 'eval_SequenceClassification_SequenceAucPr': 0.75, 'epoch': 1.0}


Model weights saved in dnabert_for_clash/checkpoint-3600/SequenceClassification/pytorch_model.bin
SequenceClassification:  97%|█████████▋| 466/481 [01:05<00:01,  7.78batches/s, epoch=30, loss=0.00971, split=train]